# TimeEvolvingMPO - Tutorial

The TimeEvolvingMPO library is currently under construction. This example serves merely as a placeholder for the tutorial to come. However, you can check that you can access the (almost) empty library by executing the `Say hi!` example below.

## Say hi!
This is just to check that you can access the tempo package.

In [ ]:
import time_evolving_mpo as tempo

In [ ]:
tempo.say_hi()